<a href="https://colab.research.google.com/github/Welle-Kim/pytorch/blob/master/%EA%B3%BC%EC%A0%9C_PyTorch%EB%A5%BC_%ED%99%9C%EC%9A%A9%ED%95%9C_fashion_mnist_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EB%A5%98_%EB%AA%A8%EB%8D%B8(DNN)_%EA%B5%AC%ED%98%84_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from tensorflow.keras.datasets import fashion_mnist
import numpy as np

# label 데이터의 각 value 에 해당하는 class name 정보
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# import 한 fashion_mnist API를 이용하여 fashion_mnist 데이터 셋을 다운로드 후 np.ndarray 객체로 메모리에 업로드
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [4]:
train_images.shape

(60000, 28, 28)

In [5]:
test_images.shape

(10000, 28, 28)

In [18]:
"""* Step 2. 데이터의 전처리"""

train_images=torch.Tensor(train_images.reshape((60000,28*28))/255.)
test_images=torch.Tensor(test_images.reshape((10000,28*28))/255.)

In [17]:
"""* Step 3. batch 구성을 위한 데이터 pipeline 생성"""

# 1. 훈련 데이터, 테스트 데이터를 파이토치의 텐서 형태로 변환해주세요 
train_x=torch.Tensor(train_images)
train_y=torch.LongTensor(train_labels)
test_x=torch.Tensor(test_images)
test_y=torch.LongTensor(test_labels)

In [9]:
# 1. torch.utils.data.TensorDataset API를 활용해 훈련, 검증, 테스트 데이터를 위한 Dataset 객체를 생성 하세요
# 2. 훈련 Dataset을 torch.utils.data.random_split() API 를 이용해 5:1로 분할하여 검증 Dataset객체를 생성 하세요

train_dataset=torch.utils.data.TensorDataset(train_x, train_y)
train_dataset,validation_dataset=torch.utils.data.random_split(train_dataset,[50000,10000])
test_dataset=torch.utils.data.TensorDataset(test_x,test_y)

In [10]:
# 1. torch.utils.data.DataLoader API를 활용해 훈련, 검증, 테스트 데이터 pipeline을 생성하세요
batch_size=128
train_loader=torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
validation_loader=torch.utils.data.DataLoader(dataset=validation_dataset,batch_size=batch_size,shuffle=False)
test_loader=torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

In [16]:
"""* Step 4. DNN(MLP) 모델 디자인"""

# 1. torch.nn.Sequential API를 이용하여 fashion_mnist 데이터 셋을 분석 하기 위한 MLP 모델을 디자인 하세요
model=torch.nn.Sequential()
model.append(torch.nn.Linear(784,256,bias=True))
model.append(torch.nn.ReLU())
model.append(torch.nn.Linear(256,10,bias=True))

Sequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)

In [14]:
"""* Step 5. 모델의 학습 정보 설정"""

# 1. torch.nn 모듈에 구현된 API를 이용해 분류를 위한 손실함수 객체를 생성하세요
# 2. torch.optim 모듈에 구현된 API로 옵티마이저를 설정하세요
loss=torch.nn.CrossEntropyLoss()
optimizer=torch.optim.RMSprop(model.parameters(),lr=1e-3)

In [24]:
"""* Step 6. 모델의 학습 loop 를 정의하는 함수 생성"""

def train(epoch):
  # 수강생 작성 코드
  # 1. 모델 객체를 학습에 적합한 상태로 변경하세요
  model.train()

  # accuracy 계산을 위한 'correct' 변수 선언 및 0 으로 초기화
  correct = 0

  # 수강생 작성 코드
  # 1. 학습데이터 에서 미니 배치를 추출하여 학습 loop를 수행하는 for문을 작성 하세요
  for batch_idx, (data, target) in enumerate(train_loader):    
    # 수강생 작성 코드
    # 1. 모델에 input data를 전달하여 순전파를 수행하세요
    output = model(data)

    # 수강생 작성 코드
    # 1. loss 함수에 모델의 예측값 과 정답 정보를 전달하여 loss 값을 계산 하세요
    batch_loss = loss(output, target)

    # 수강생 작성 코드
    # 1. optimizer 객체를 이용해 모델을 구성하는 파라미터들(w, b)의 gradient를 초기화 하세요
    optimizer.zero_grad()

    # 수강생 작성 코드
    # 1. 모델을 구성하는 파라미터들(w, b)이 loss 에 미치는 영향도(gradient)를 계산 하세요
    batch_loss.backward()
    
    # 수강생 작성 코드
    # 1. 계산된 영향도(gradient) 값을 이용하여 모델의 파라미터(w,b)를 업데이트 하세요
    optimizer.step()

    # 모델의 성능 지표를 계산 및 출력 하여 학습 진행을 체크하는 코드
    correct += (output.argmax(dim=1) == target).type(torch.float).sum().item()

    if batch_idx%50==0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAccuracy: {:.2f}%'.format(
            epoch,
            (batch_idx+1) * len(data),
            len(train_loader.dataset),
            100. * (batch_idx+1) / len(train_loader),
            batch_loss.item(),
            100. * correct / ((batch_idx+1) * len(data))
            ))
    

"""* Step 7. 모델의 검증 loop 를 정의하는 함수 생성"""

def test(data_loader):
  # 수강생 작성 코드
  # 1. 모델 객체를 테스트에 적합한 상태로 변경하세요
  model.eval()
  
  # 수강생 작성 코드
  # 1. 모델의 성능을 지표를 계산에 필요한 정보 수집을 위한 변수 선언
  #   - loss 를 수집하기 위한 'test_loss' 변수 선언 및 0 으로 초기화
  #   - accuracy 계산을 위한 모델의 예측 class 와 정답 class 가 일치하는 개수를 수집하기 위한
  #     'correct' 변수 선언 및 0 으로 초기화
  test_loss = 0
  correct = 0

  # 수강생 작성 코드
  # 1. 검증데이터 에서 미니 배치를 추출하여 검증 loop를 수행하는 for문을 작성 하세요  
  for data, target in data_loader:
    # 수강생 작성 코드
    # 1. 모델에 input data를 전달하여 순전파를 수행하세요
    output = model(data)
    
    # 수강생 작성 코드
    # 1. loss 함수에 모델의 예측값 과 정답 정보를 전달하여 모델의 성능을 검증하기 위한 loss 값을 계산 하세요
    test_loss += loss(output, target).item()

    # 수강생 작성 코드
    # 1. logits 정보를 이용하여 모델의 예측 값과 정답 을 비교하여 모델이 맞춘 데이터의 개수를 계산 하세요
    #   - 맞춘 데이터의 개수는 correct 변수에 누적해서 더하세요
    correct += (output.argmax(dim=1) == target).type(torch.float).sum().item()

  # 수강생 작성 코드
  # 1. 'test_loss' 변수 에 누적된 배치 별 loss 의 합을 이용해 전체 loss를 계산 하세요
  #   - 누적된 배치 별 loss 의 합을 배치의 개수로 나누에 전체 데이터 셋의 평균 loss를 계산
  #   - 계산 된 평균 loss를 할당 연산자를 이용하여 'test_loss' 할당
  test_loss /= len(test_loader)
  
  # 수강생 작성 코드
  # 1. 'correct' 에 누적된 배치 별 정답 개수의 합을 배치의 개수로 나누에 전체 데이터 셋의 평균 accuracy를 계산 하세요
  #    - 누적된 배치 별 정답 개수의 합을 전체 데이터 개수로 나누에 전체 데이터 셋의 평균 accuracy를 계산
  #    - 계산 된 평균 accuracy를 'accuracy' 변수에 할당
  accuracy = 100. * correct / len(validation_loader.dataset)
  
  # 모델의 성능 지표를 계산 및 출력 하여 학습 진행을 체크하는 코드
  print('\nTest set: Average loss: {:.4f}, Accuracy: {:.0f}%\n'.format(
        test_loss,
        accuracy))

"""* Step 8. 훈련 함수와 검증 함수를 이용한 모델의 훈련 및 검증"""
# 수강생 작성 코드
# 1. 반복문을 활용해 epoch 수 만큼 훈련 및 검증을 수행 하세요.
epochs = 30
for epoch in range(1, epochs):
  train(epoch)
  test(validation_loader)

# 수강생 작성 코드
# 1. 테스트 데이터 셋을 이용해 모델의 성능을 확인 하세요
test(test_loader)

Train Epoch: 1 [128/50000 (0%)]	Loss: 2.296693	Accuracy: 12.50%
Train Epoch: 1 [6528/50000 (13%)]	Loss: 2.289948	Accuracy: 10.34%
Train Epoch: 1 [12928/50000 (26%)]	Loss: 2.311039	Accuracy: 10.34%
Train Epoch: 1 [19328/50000 (39%)]	Loss: 2.300928	Accuracy: 10.25%
Train Epoch: 1 [25728/50000 (51%)]	Loss: 2.313652	Accuracy: 10.28%
Train Epoch: 1 [32128/50000 (64%)]	Loss: 2.300995	Accuracy: 10.31%
Train Epoch: 1 [38528/50000 (77%)]	Loss: 2.315243	Accuracy: 10.29%
Train Epoch: 1 [44928/50000 (90%)]	Loss: 2.318026	Accuracy: 10.34%

Test set: Average loss: 2.3141, Accuracy: 10%

Train Epoch: 2 [128/50000 (0%)]	Loss: 2.311678	Accuracy: 12.50%
Train Epoch: 2 [6528/50000 (13%)]	Loss: 2.303497	Accuracy: 9.87%
Train Epoch: 2 [12928/50000 (26%)]	Loss: 2.310434	Accuracy: 10.38%
Train Epoch: 2 [19328/50000 (39%)]	Loss: 2.303534	Accuracy: 10.17%
Train Epoch: 2 [25728/50000 (51%)]	Loss: 2.324116	Accuracy: 10.23%
Train Epoch: 2 [32128/50000 (64%)]	Loss: 2.325551	Accuracy: 10.21%
Train Epoch: 2 [38528/5

All copyright by codepresso